In [1]:
##Gunnar Dulle
##CS 340
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aac"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read_all({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    #Adds the Gravioso Salvare Logo and formats it to be smaller and align right
    html.A([html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),style={
        'height':'15%',
        'width':'15%',
        'float':'right',
        'position':'relative',
        'padding-top':0,
        'padding-right':0
    })],
          href='www.snhu.edu'),
    #My name added to Dashboard label in order to get all aspects of dashboard to fit in one screenshot
    html.Center(html.B(html.H1('Gunnar Dulle SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        #Radio items added for data filters
        dcc.RadioItems(
        id='filter-type',
        options=[
            {'label' : 'Water Rescue', 'value' :  'WR'},
            {'label' : 'Mountain or Wilderness Rescue', 'value' : 'MWR'},
            {'label' : 'Disaster Rescue or Individual Tracking', 'value' : 'DIT'},
            {'label' : 'Reset', 'value' : 'R'}
        ],
        value = 'R',
        labelStyle = {'display' : 'inline-block'}
        ),

    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = "single",
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries
    #Filter for Reset
    if filter_type == 'R':
        new_df = df
    #Filter for Water Rescue as no dogs with breed Newfoundland existed in the database I added all variants of Newfoundland to this filter
    elif filter_type == 'WR':
        new_df = pd.DataFrame.from_records(shelter.read_all({'$and':[{'animal_type':'Dog'},
                                                                    {'$or':[
                                                                        {'breed':'Labrador Retriever Mix'},
                                                                        {'breed':'Chesa Bay Retr Mix'},
                                                                        {'breed':'Newfoundland Mix'},
                                                                        {'breed':'Newfoundland/Australian Cattle Dog'},
                                                                        {'breed':'Newfoundland/Great Pyrenees'},
                                                                        {'breed':'Newfoundland/Labrador Retriever'}]
                                                                    },
                                                                    {'$and':[{'sex_upon_outcome':'Intact Female'},
                                                                            {'age_upon_outcome_in_weeks':{'$gte':26}},
                                                                            {'age_upon_outcome_in_weeks':{'$lte':156}}]
                                                                    }]
                                                            }))
    #Filter for Mountain or Wilderness Rescue
    elif filter_type == 'MWR':
        new_df = pd.DataFrame.from_records(shelter.read_all({'$and':[{'animal_type':'Dog'},
                                                                    {'$or':[
                                                                        {'breed':'German Shepherd'},
                                                                        {'breed':'Alaskan Malamute'},
                                                                        {'breed':'Old English Sheepdog'},
                                                                        {'breed':'Siberian Husky'},
                                                                        {'breed':'Rottweiler'}]
                                                                    },
                                                                    {'$and':[{'sex_upon_outcome':'Intact Male'},
                                                                            {'age_upon_outcome_in_weeks':{'$gte':26}},
                                                                            {'age_upon_outcome_in_weeks':{'$lte':156}}]
                                                                    }]
                                                            }))
    #Filter for Disaster Rescue or Individual Tracking
    elif filter_type == 'DIT':
        new_df = pd.DataFrame.from_records(shelter.read_all({'$and':[{'animal_type':'Dog'},
                                                                    {'$or':[
                                                                        {'breed':'Doberman Pinsch'},
                                                                        {'breed':'German Shepherd'},
                                                                        {'breed':'Golden Retriever'},
                                                                        {'breed':'Bloodhound'},
                                                                        {'breed':'Rottweiler'}]
                                                                    },
                                                                    {'$and':[{'sex_upon_outcome':'Intact Male'},
                                                                            {'age_upon_outcome_in_weeks':{'$gte':20}},
                                                                            {'age_upon_outcome_in_weeks':{'$lte':300}}]
                                                                    }]
                                                            }))
    #Start with filter set on Reset
    else:
        new_df = df

        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in new_df.columns]
    data=new_df.to_dict('records')
        
        
    return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    #Imports currently displayed data only displays data for information on the current page
    dff = pd.DataFrame.from_dict(viewData)
    #holds data for names(breeds) and values(recurring counts)
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    
    #Return updated graph data
    return [
        dcc.Graph(
            figure = px.pie(data_frame = dff, values = values, names = names)
        )
    ]
    

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):                           
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            #Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[0,13],dff.iloc[0,14]], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H4("Animal Name"),
                    html.P(dff.iloc[0,9])
                ])
            ])
        ])
    ]

app